## 생성 모델

- 주어진 데이터를 학습하여 **데이터 분포**를 따르는 유사한 데이터를 생성하는 모델

- 판별자 : 대상의 **특징**들을 찾아내서 이미지를 분류할 수 있도록 함

  - ex) 강아지/고양이 분류를 위해 눈, 꼬리 등의 특성을 파악함
<br><br>
- 생성자 : 판별자에서 추출한 특징을 이용하여 새로운 이미지를 생성하도록 함

  - 입력 이미지에 대한 **데이터 분포**를 학습하여, 입력 이미지와 유사한 이미지를 생성하도록 함

- 명시적 방법

  - 모델의 확률 변수 p(x)를 **정의**하여 사용함

  - ex) VAE(Variational AutoEncoder)
<br><br>
- 암시적 방법

  - 확률 변수에 대한 정의 없이 p(x)를 **샘플링**하여 사용함
  
  - ex) GAN(Generative Adversarial Network)

---

## 변형 오토인코더

#### AutoEncoder

- AutoEncoder의 은닉층의 뉴런 수는 입력층과 출력층의 뉴런 수보다 훨씬 적음
  
  -> 은닉층의 **적은 뉴런**으로도 데이터를 가장 잘 표현할 수 있게 하는 방법

- 인코더 : `특성`에 대한 학습을 수행함 (recognition network)

- 은닉층 : 차원이 가장 낮은, 입력 데이터의 `압축된 표현`이 만들어짐

- 디코더 : 압축된 데이터를 원래대로 `복원`함 (generative network)

- 손실 재구성 : 손실함수는 {입력값}과 {복원된 출력값}의 차이로 계산됨

<img src="https://www.jeremyjordan.me/content/images/2018/03/Screen-Shot-2018-03-06-at-3.17.13-PM.png" width="500px">

- 입력 x를 인코더 h에 통과시켜, 압축된 잠재 벡터 z를 얻음

  - z = h(x)
<br><br>
- 압축된 z 벡터를 디코더 g에 통과시켜, 출력 y를 얻음

  - y = g(z) = g(h(x))
<br><br>
- 이 때, 출력 y는 입력 x와 크기가 같아야 함
<br><br>
- 손실함수 값은 입력 x와 출력 y의 차이로 계산됨

<img src="https://thebook.io/img/080289/685.jpg" width="500px">

- 데이터 압축 : 데이터의 중요 특성만 압축하여 메모리 측면에서 이점을 얻음
<br><br>
- 차원의 저주 방지 : 특성의 개수를 줄여주어(데이터 차원을 감소시켜) 차원의 저주를 피할 수 있음
<br><br>
- 특성 추출 : 비지도 학습으로, 주요 특성을 자동으로 찾아줌

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms

from torchsummary import summary

In [2]:
class Encoder(nn.Module):
    def __init__(self, encoded_space_dim, fc2_input_dim):
        super().__init__()

        # Convolution 연산으로 이미지 특성 추출 (차원 축소)
        self.encoder_cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=0),
            nn.ReLU(inplace=True)
        )

        self.flatten = nn.Flatten(start_dim=1)

        # 잠재벡터 생성
        self.encoder_lin = nn.Sequential(
            nn.Linear(3*3*32, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, encoded_space_dim)
        )


    def forward(self, x):
        x = self.encoder_cnn(x)
        x = self.flatten(x)
        x = self.encoder_lin(x)
        return x

In [3]:
class Decoder(nn.Module):
    def __init__(self, encoded_space_dim, fc2_input_dim):
        super().__init__()

        # 인코더의 출력을 디코더의 입력으로 사용
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 3*3*32),
            nn.ReLU(inplace=True)
        )


        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(32, 3, 3))

        # 이미지 복원
        self.decoder_cnn = nn.Sequential(
            nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, output_padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=3, stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(in_channels=8, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True)
        )


    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

In [4]:
encoder = Encoder(encoded_space_dim=4, fc2_input_dim=128)
decoder = Decoder(encoded_space_dim=4, fc2_input_dim=128)

In [5]:
encoder

Encoder(
  (encoder_cnn): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (6): ReLU(inplace=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (encoder_lin): Sequential(
    (0): Linear(in_features=288, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=4, bias=True)
  )
)

In [6]:
decoder

Decoder(
  (decoder_lin): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=288, bias=True)
    (3): ReLU(inplace=True)
  )
  (unflatten): Unflatten(dim=1, unflattened_size=(32, 3, 3))
  (decoder_cnn): Sequential(
    (0): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(8, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (7): ReLU(inplace=True)
  )
)

In [8]:
summary(encoder.to(torch.device('cuda')), input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 14, 14]              80
              ReLU-2            [-1, 8, 14, 14]               0
            Conv2d-3             [-1, 16, 7, 7]           1,168
       BatchNorm2d-4             [-1, 16, 7, 7]              32
              ReLU-5             [-1, 16, 7, 7]               0
            Conv2d-6             [-1, 32, 3, 3]           4,640
              ReLU-7             [-1, 32, 3, 3]               0
           Flatten-8                  [-1, 288]               0
            Linear-9                  [-1, 128]          36,992
             ReLU-10                  [-1, 128]               0
           Linear-11                    [-1, 4]             516
Total params: 43,428
Trainable params: 43,428
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

#### Variational AutoEncoder

- 일반적인 AutoEncoder

  - 입력 -> 인코더 -> 압축 -> 디코더 -> 출력

  - 차원을 축소하고 `입력과 동일한 데이터`를 만드는 것이 목표
  
    - 새로 생성된 데이터의 확률 분포에는 관심 없음

- Variational AutoEncoder

  - 평균과 표준편차를 이용하여 `확률 분포`를 만듦

  - 여기서 샘플링하여 디코더를 통과시킨 후, `입력과 조금 다른 데이터`를 만들어냄
  
  - 데이터가 만들어지는 확률 분포를 찾아 유사한 데이터를 생성하는 것이 목표

<img src="https://vitalflux.com/wp-content/uploads/2023/04/autoencoder-vs-variational-autoencoder-point-vs-distribution.png" width="350px">

##### Encoder

- 데이터 x가 주어졌을 때, 디코더가 잘 복원할 수 있는 **이상적인 확률 분포 p(z|x)** 찾는 것이 목표

- 이상적인 확률 분포 찾는 데는 **변분추론(variational inference)** 사용

  - `가우시안 분포`를 가정하고 모수를 바꿔가며, 이상적 확률 분포에 근사하게 만들어 사용하는 것

In [9]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.input1 = nn.Linear(input_dim, hidden_dim)
        self.input2 = nn.Linear(hidden_dim, hidden_dim)
        self.mean = nn.Linear(hidden_dim, latent_dim)
        self.var = nn.Linear(hidden_dim, latent_dim)

        self.LeakyReLU = nn.LeakyReLU(0.2)
        self.training = True


    def forward(self, x):
        h_ = self.LeakyReLU(self.input1(x))
        h_ = self.LeakyReLU(self.input2(h_))
        mean = self.mean(h_)
        log_var = self.var(h_)
        return mean, log_var # 평균, 분산을 반환

##### Decoder

- 추출한 샘플을 입력받아 원본으로 **복원**하는 것이 목표

In [10]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.hidden1 = nn.Linear(latent_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.LeakyReLU = nn.LeakyReLU(0.2)


    def foward(self, x):
        h = self.LeakyReLU(self.hidden1(x))
        h = self.LeakyReLU(self.hidden2(h))
        x_hat = torch.sigmoid(self.output(h))
        return x_hat

##### VAE

In [11]:
class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder

    # 잠재변수 z 샘플링 함수
    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var)
        z = mean + var * epsilon
        return z

    def forward(self, x):
        mean, log_var = self.Encoder(x) # 인코더에서 평균, 표준편차를 받아옴
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # 평균, 표준편차를 이용해 z를 생성함 (z는 가우시안 분포로 가정했기 때문에)
        x_hat = self.Decoder(z) # z를 디코더에 통과시켜, 입력 x와 유사한 x^ 생성함
        return x_hat, mean, log_var

In [12]:
x_dim = 784
hidden_dim = 400
latent_dim = 200

encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim=hidden_dim, output_dim=x_dim)

model = Model(Encoder=encoder, Decoder=decoder)

In [13]:
model

Model(
  (Encoder): Encoder(
    (input1): Linear(in_features=784, out_features=400, bias=True)
    (input2): Linear(in_features=400, out_features=400, bias=True)
    (mean): Linear(in_features=400, out_features=200, bias=True)
    (var): Linear(in_features=400, out_features=200, bias=True)
    (LeakyReLU): LeakyReLU(negative_slope=0.2)
  )
  (Decoder): Decoder(
    (hidden1): Linear(in_features=200, out_features=400, bias=True)
    (hidden2): Linear(in_features=400, out_features=400, bias=True)
    (output): Linear(in_features=400, out_features=784, bias=True)
    (LeakyReLU): LeakyReLU(negative_slope=0.2)
  )
)